In [26]:
import pandas as pd
import numpy as np
from scipy import stats
from models import get_pvalue

In [8]:
test_demographics = pd.read_csv('../data/raw/s4/demo.csv', parse_dates=['reg_date'], infer_datetime_format=True)
test_results = pd.read_csv('../data/raw/s4/ab_test_results.csv', parse_dates=['date'], infer_datetime_format=True)

In [9]:
test_demographics.head()

,uid,reg_date,device,gender,country,age
0,52774929.0,2018-03-07,and,F,FRA,27
1,84341593.0,2017-09-22,iOS,F,TUR,22
2,41201055.0,2017-11-24,and,F,USA,20
3,68477880.0,2016-12-08,and,F,BRA,18
4,42235969.0,2017-09-30,and,F,USA,41


In [10]:
# Group column: C for control; V for variant
test_results.head()

,uid,date,purchase,sku,price,group
0,82082526.0,2018-02-15 06:24:30,0,NaN,NaN,V
1,82082526.0,2018-02-16 21:52:38,0,NaN,NaN,V
2,82082526.0,2018-02-17 03:17:59,0,NaN,NaN,V
3,82082526.0,2018-02-18 18:16:27,0,NaN,NaN,V
4,82082526.0,2018-02-19 02:12:14,0,NaN,NaN,V


In [11]:
test_results.groupby(by='group', as_index=False).uid.count()

,group,uid
0,C,58583
1,V,56350


In [13]:
test_result_demo = test_results.merge(test_demographics, how='inner', on='uid')

test_result_demo.groupby(by=['country', 'gender', 'device','group'], as_index=False).uid.count()

,country,gender,device,group,uid
0,BRA,F,and,C,5070
1,BRA,F,and,V,4136
2,BRA,F,iOS,C,3359
3,BRA,F,iOS,V,2817
4,BRA,M,and,C,3562
5,BRA,M,and,V,3673
6,BRA,M,iOS,C,2940
7,BRA,M,iOS,V,3109
8,CAN,F,and,C,747
9,CAN,F,and,V,806


In [23]:
test_result_group = test_result_demo.groupby(by='group', as_index=False).agg({'purchase':['count','sum']})
test_result_group.columns = test_result_group.columns.droplevel(0)
test_result_group['conv'] = test_result_group['sum'] / test_result_group['count']
test_result_group.head()

,,count,sum,conv
0,C,58583,5329,0.090965
1,V,56350,5748,0.102005


In [41]:
#48236   1657    0.034351
#49867   2094    0.041984

p1 = 0.034351
q1 = (1-p1)
p2 = 0.041984
q2 = (1-p2)
m = 48236 
n = 49867

p_hat = (m * p1 + n * p2) / (m + n)
q_hat = (1-p_hat)

z1 = 2 * ((p1 - p2) / (p_hat*q_hat*(1/m+1/n)) ** 0.5)
z2 = 2 * (p1 - p2) / ((p1 * q1 / m + p2 * q2 / n ) ** 0.5)
print(stats.norm.cdf(z1, loc=0))
print(stats.norm.cdf(z2, loc=0))

5.7083835187050216e-36
4.3089120024502883e-36


In [38]:
get_pvalue(p1,p2,m,n)

4.257297485586909e-10